# Proyecto clustering

## Análisis preliminar

1. Columnas con valores nulos
   - homePage tiene muchos valores nulos (5807), por lo que no es útil para el clustering.
   - video tiene 486 valores nulos y no aporta a la agrupación.
   - productionCompany, productionCompanyCountry, productionCountry, directors, actors y actorsPopularity tienen algunos valores nulos, pero pueden ser útiles tras limpieza.

2. Variables irrelevantes para clustering
   - id: es solo un identificador.
   - homePage: no aporta al análisis.
   - video: no es relavante para el clustering.
   - originalTitle y title: son solo nombres.
   - releaseDate: como dato puntual no ayuda, pero se podría extraer el año para análisis.
   

1. Clustering

    1.1 Haga el preprocesamiento del dataset, explique qué variables no aportan información a la
        generación de grupos y por qué. Describa con qué variables calculará los grupos.

       Variables irrelevantes para clustering:
           - id: es solo un identificador
           - title: es textual
           - original_title: es textual
           - homePage: es textual
           - video: es textual
           - actorsCharacter: es textual
           - releaseDate: 

       Variables útiles para clustering:
            - popularity

In [27]:
import pandas as pd
import numpy as np

# 1. Cargar archivo de datasets
df = pd.read_csv("movies.csv", encoding="ISO-8859-1")

# PRUEBA 1 = DA VALOR DE HOPKINS DE 0.009424485458607145
# Columnas a eliminar
# columns_to_drop = ["id", "homePage", "video", "originalTitle", "title", "releaseDate", "actorsCharacter"]
# Eliminar columnas que no aportan información
# df = df.drop(columns=columns_to_drop)


# Mostramos que columnas tienen muchos valores nulos para excluirlos
#print(df.isnull().sum().sort_values(ascending=False))

# Eliminar filas con valores nulos (si es necesario)
# df = df.dropna()

# PRUEBA 2
numerical_features = ["budget", "revenue", "popularity", "voteCount", "voteAvg", "runtime"]
df_numerical = df[numerical_features].dropna()

# Eliminar filas con valores nulos
# df = df.dropna(subset=["budget", "revenue", "popularity", "voteCount", "voteAvg", "runtime"])

    1.2 Analice la tendencia al agrupamiento usando el estadístico de Hopkings y la VAT (Visual
        Assessment of cluster Tendency). Esta última hágala si es posible, teniendo en cuenta las
        dimensiones del conjunto de datos. Discuta sus resultados e impresiones.

In [18]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.8 kB 2.6 MB/s eta 0:00:01
     ---------------------------------------- 60.8/60.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.2/11.1 MB 3.3 MB/s eta 0:00:04
   - -------------------------------------- 0.4/11.1 MB 3.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.1 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/11.1 MB 5.6 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.1 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.1 MB 6.3 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.1 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.5/11.1 MB 6.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.7/11.1 MB 6.7 MB/s eta 0:00:02
   --


[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform


def hopkins_statistic(X, sample_size=0.1):
    X = X.values if isinstance(X, pd.DataFrame) else X
    n, d = X.shape
    sample_n = max(1, int(sample_size * n))

    random_indices = sample(range(n), sample_n)
    X_real_samples = X[random_indices]

    X_synthetic_samples = np.array([uniform(X[:, col].min(), X[:, col].max(), sample_n) for col in range(d)]).T

    nbrs = NearestNeighbors(n_neighbors=2).fit(X)
    
    real_distances = np.array([nbrs.kneighbors([x], 2, return_distance=True)[0][0][1] for x in X_real_samples])
    synthetic_distances = np.array([nbrs.kneighbors([x], 1, return_distance=True)[0][0][0] for x in X_synthetic_samples])

    H = synthetic_distances.sum() / (synthetic_distances.sum() + real_distances.sum())
    return H

# Calcular el estadístico de Hopkins
# X = df.select_dtypes(include=[np.number]).values
# hopkins = hopkins_statistic(X)

# PRUEBA 2
hopkins_value = hopkins_statistic(df_numerical)
print(f"\n🔹 Estadística de Hopkins: {hopkins_value:.4f}")


🔹 Estadística de Hopkins: 0.9878
